In [1]:
import sys
import copy
import math
import pickle

sys.path.append("../../")

import numpy as np
import torch
import matplotlib.pyplot as plt

import implicitmodules.torch as dm

torch.set_default_tensor_type(torch.DoubleTensor)

In [28]:
def sample(pts_silent, pts_implicit1, R, C, sigma):
    # We first pick a random index in pts_silent and get the normal position
    # We substract 1 to the possible index so we can easily get the tangent vector
    # ind = np.asscalar(np.random.choice(pts_silent.shape[0] - 1, 1))
    # tan = (pts_silent[ind + 1] - pts_silent[ind]).numpy()
    # norm = np.array([tan[1], -tan[0]])
    # norm /= np.linalg.norm(norm)
    # norm *= -150.
    # cotan = torch.zeros_like(pts_silent)
    # cotan[ind] = torch.tensor(norm)

    #cotan = 10.*torch.randn_like(pts_silent)
    cotan = torch.zeros_like(pts_silent)
    # cotan[:10, 0] = -10.
    # cotan[120:130, 0] = -10.

    val_left = -1.
    val_right = 1.
    cotan[30:39, 1] = val_left
    cotan[212:221, 1] = -val_left
    cotan[92:101, 1] = val_right
    cotan[149:158, 1] = -val_right

    #plt.plot(pts_silent[:, 0].numpy(), pts_silent[:, 1].numpy())
    #plt.quiver(pts_silent[:, 0].numpy(), pts_silent[:, 1].numpy(), cotan[:, 0].numpy(), cotan[:, 1].numpy())

    silent = dm.DeformationModules.SilentLandmarks.build_from_points(pts_silent)
    silent.manifold.fill_cotan(cotan.view(-1).requires_grad_())
    # implicit1 = dm.DeformationModules.ImplicitModule1.build_and_fill(2, pts_implicit1.shape[0], C[:, :, 0].unsqueeze(2), sigma, 0.001, gd=(pts_implicit1.view(-1).requires_grad_(), R.view(-1).requires_grad_()))
    # implicit1_2 = dm.DeformationModules.ImplicitModule1.build_and_fill(2, pts_implicit1.shape[0], C[:, :, 1].unsqueeze(2), sigma, 0.001, gd=(pts_implicit1.view(-1).requires_grad_(), R.view(-1).requires_grad_()))
    implicit1 = dm.DeformationModules.ImplicitModule1.build_and_fill(2, pts_implicit1.shape[0], C, sigma, 0.001, gd=(pts_implicit1.view(-1).requires_grad_(), R.view(-1).requires_grad_()))
    with torch.autograd.no_grad():
        _, controls = dm.HamiltonianDynamic.shoot(dm.HamiltonianDynamic.Hamiltonian([silent, implicit1]), 500, 'midpoint', intermediates=True)

    for l in controls:
        print(l)

    return silent.manifold.gd.detach().view(-1, 2)

In [17]:
peanut = torch.tensor(pickle.load(open("../../data/peanuts.pickle", 'rb'))[0][0]).type(torch.get_default_dtype())
print(peanut.shape)
aabb_peanut = dm.Utilities.AABB.build_from_points(peanut)

density = 100

pts_implicit1 = dm.Utilities.fill_aabb(aabb_peanut, density)

print(aabb_peanut.area*density)
print(pts_implicit1.shape[0])

torch.Size([241, 2])
tensor(506.4226)
476


In [18]:
plt.plot(peanut.numpy()[:-1, 0], peanut.numpy()[:-1, 1])
plt.plot(pts_implicit1.numpy()[:, 0], pts_implicit1.numpy()[:, 1], '.')
plt.plot(peanut[30:39, 0].numpy(), peanut[30:39, 1].numpy(), 'r*')
plt.plot(peanut[92:101, 0].numpy(), peanut[92:101, 1].numpy(), 'r*')
plt.plot(peanut[149:158, 0].numpy(), peanut[149:158, 1].numpy(), 'r*')
plt.plot(peanut[212:221, 0].numpy(), peanut[212:221, 1].numpy(), 'r*')
plt.axis('equal')
plt.show()

In [20]:
C = torch.zeros(pts_implicit1.shape[0], 2, 2)
# C[pts_implicit1[:, 0] >= 0, 0, 0] = -10.
# # C[pts_implicit1[:, 0] >= 0, 1, 0] = 5.
# # C[pts_implicit1[:, 0] < 0, 0, 1] = 20.
# # C[pts_implicit1[:, 0] < 0, 1, 1] = 8.
# C[pts_implicit1[:, 0] > 0.25, 0, 1] = 1.
# C[pts_implicit1[:, 0] > 0.25, 1, 1] = 1.
# C[pts_implicit1[:, 0] < -0.25, 0, 0] = 1.
# C[pts_implicit1[:, 0] < -0.25, 1, 0] = 1.
C[pts_implicit1[:, 0] >= 0., 0, 1] = 1.
C[pts_implicit1[:, 0] >= 0., 1, 1] = 1.
C[pts_implicit1[:, 0] < 0., 0, 0] = 1.
C[pts_implicit1[:, 0] < 0., 1, 0] = 1.

%matplotlib qt5
ax = plt.subplot()
dm.Utilities.plot_C_ellipse(ax, pts_implicit1, C, c_index=0, scale=0.1, color='red')
dm.Utilities.plot_C_ellipse(ax, pts_implicit1, C, c_index=1, scale=0.1, color='blue')
dm.Utilities.plot_C_arrow(ax, pts_implicit1, C, c_index=0, scale=0.1, color='black', zorder=3, mutation_scale=10)
dm.Utilities.plot_C_arrow(ax, pts_implicit1, C, c_index=1, scale=0.1, color='black', zorder=3, mutation_scale=10)
plt.axis(aabb_peanut.get_list())
plt.axis('equal')
plt.show()

In [29]:
angles = 0. * math.pi * torch.ones(pts_implicit1.shape[0])
R = torch.stack([dm.Utilities.rot2d(a) for a in angles])

my_sample = sample(peanut, pts_implicit1, R, C, 0.1)

ax = plt.subplot()
plt.plot(peanut[:, 0].numpy(), peanut[:, 1].numpy(), '--')
plt.plot(my_sample[:, 0].numpy(), my_sample[:, 1].numpy())
plt.axis('equal')
plt.show()

tensor(0.1847)


tensor(0.1847)


tensor(0.1847)


tensor(0.1847)


tensor(0.1847)


...

tensor(0.1858)


tensor(0.1860)


tensor(0.1858)


tensor(0.1860)


[tensor([]), tensor([-0.0136,  0.0135])]
[tensor([]), tensor([-0.0136,  0.0134])]
[tensor([]), tensor([-0.0136,  0.0134])]
[tensor([]), tensor([-0.0137,  0.0134])]
[tensor([]), tensor([-0.0137,  0.0134])]
[tensor([]), tensor([-0.0137,  0.0134])]
[tensor([]), tensor([-0.0137,  0.0133])]
[tensor([]), tensor([-0.0137,  0.0133])]
[tensor([]), tensor([-0.0138,  0.0133])]
[tensor([]), tensor([-0.0138,  0.0133])]
[tensor([]), tensor([-0.0138,  0.0132])]
[tensor([]), tensor([-0.0138,  0.0132])]
[tensor([]), tensor([-0.0139,  0.0132])]
[tensor([]), tensor([-0.0139,  0.0132])]
[tensor([]), tensor([-0.0139,  0.0132])]
[tensor([]), tensor([-0.0139,  0.0131])]
[tensor([]), tensor([-0.0139,  0.0131])]
[tensor([]), tensor([-0.0140,  0.0131])]
[tensor([]), tensor([-0.0140,  0.0131])]
[tensor([]), tensor([-0.0140,  0.0130])]
[tensor([]), tensor([-0.0140,  0.0130])]
[tensor([]), tensor([-0.0140,  0.0130])]
[tensor([]), tensor([-0.0141,  0.0130])]
[tensor([]), tensor([-0.0141,  0.0130])]
[tensor([]), ten

In [7]:
sample_count = 40
samples = [peanut.tolist()]
for i in range(sample_count):
    print("Generating sample", i)
    samples.append(sample(peanut, pts_implicit1, R, C, 0.4).tolist())

Generating sample 0


KeyboardInterrupt: 

In [6]:
# Does not work somehow

nb_plot_row = 5
nb_plot_col = 5
cur_plot_index = 0
while True:
    for i in range(nb_plot_row):
        for j in range(nb_plot_col):
            if cur_plot_index < len(samples):
                plt.subplot(nb_plot_row, nb_plot_col, nb_plot_row*i+j+1)
                plt.plot(np.asarray(samples[cur_plot_index])[:, 0], np.asarray(samples[cur_plot_index])[:, 1])
                cur_plot_index = cur_plot_index + 1
            else:
                plt.show()
                break

    plt.show()        

NameError: name 'samples' is not defined

In [9]:
pickle.dump({'c_gt_pos': pts_implicit1, 'c_gt': C, 'dataset': samples}, open("../../data/deformed_peanuts.pkl", 'wb'))